# Session 5: AI Agents with Memory

**Duration**: 90 minutes  
**Difficulty**: Intermediate to Advanced

## Learning Objectives

- 🎯 Understand AI agent architectures  
- 🎯 Implement the ReAct (Reasoning + Acting) pattern  
- 🎯 Build agent execution loops  
- 🎯 Create memory systems (short-term, long-term)  
- 🎯 Handle autonomous multi-step tasks  
- 🎯 Make SupportGenie truly autonomous

## 📚 What You'll Build

**SupportGenie v0.5**: Autonomous Agent with Memory
- Autonomous task planning
- Conversation memory  
- Customer profile memory
- Multi-step problem solving
- Learns from past interactions

## Part 0: Setup

In [ ]:
# Install required packages
!pip install openai python-dotenv -q

print("✅ Packages installed!")

In [ ]:
import os
import json
import random
from datetime import datetime
from openai import OpenAI

# Set up API key
try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    print("✅ API key loaded from Colab secrets")
except:
    from getpass import getpass
    if 'OPENAI_API_KEY' not in os.environ:
        os.environ['OPENAI_API_KEY'] = getpass('Enter your OpenAI API key: ')
    print("✅ API key loaded")

# Initialize client
client = OpenAI(api_key=os.environ.get('OPENAI_API_KEY'))

print("\n🚀 Ready to build agents!")

## Part 1: What is an AI Agent?

### Definition

An **AI Agent** is a system that:
1. **Observes** the environment/user input
2. **Thinks** about what to do (reasoning)
3. **Acts** using tools/functions
4. **Remembers** context and history
5. **Iterates** until task complete

### Agent vs Chatbot

**Chatbot** (What we built so far):
- User asks → LLM responds
- Single turn interaction
- No persistent memory beyond conversation

**Agent**:
- User gives task → Agent plans → Uses tools → Iterates → Completes task
- Multi-turn, autonomous
- Remembers past interactions

## Part 2: The ReAct Pattern

### ReAct = Reasoning + Acting

The agent alternates between thinking and acting:

```
Thought: What do I need to do?
Action: [Use a tool]
Observation: [Tool result]
Thought: Based on the observation, what next?
Action: [Use another tool]
Observation: [Result]
Thought: Task complete!
Final Answer: [Response to user]
```

## Part 3: Simple Agent Execution Loop

Let's build a basic agent that can execute multiple tool calls.

In [ ]:
# Define some tools for our agent

def get_current_date() -> str:
    """Get today's date"""
    return datetime.now().strftime("%Y-%m-%d")

def search_knowledge_base(query: str) -> dict:
    """Search knowledge base (mock)"""
    # Mock knowledge base
    kb = {
        "refund policy": "Refunds available within 30 days of purchase.",
        "shipping": "Free shipping on orders over $50. Standard delivery 5-7 days.",
        "warranty": "All products come with 1-year warranty."
    }
    
    for key, value in kb.items():
        if key in query.lower():
            return {"result": value, "source": "knowledge_base"}
    
    return {"result": "No information found", "source": None}

def calculate(expression: str) -> dict:
    """Calculate mathematical expression"""
    try:
        # Safe eval for simple math
        result = eval(expression, {"__builtins__": {}}, {})
        return {"result": result}
    except:
        return {"error": "Invalid expression"}

# Tool definitions
agent_tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_date",
            "description": "Get today's date",
            "parameters": {"type": "object", "properties": {}}
        }
    },
    {
        "type": "function",
        "function": {
            "name": "search_knowledge_base",
            "description": "Search the knowledge base for information",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {"type": "string", "description": "Search query"}
                },
                "required": ["query"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "calculate",
            "description": "Calculate a mathematical expression",
            "parameters": {
                "type": "object",
                "properties": {
                    "expression": {"type": "string", "description": "Math expression like '2+2' or '10*5'"}
                },
                "required": ["expression"]
            }
        }
    }
]

# Map function names to implementations
available_functions = {
    "get_current_date": get_current_date,
    "search_knowledge_base": search_knowledge_base,
    "calculate": calculate
}

print("✅ Agent tools defined")

In [ ]:
class SimpleAgent:
    """Basic agent with execution loop"""
    
    def __init__(self, tools, max_iterations=10):
        self.client = OpenAI()
        self.tools = tools
        self.max_iterations = max_iterations
        self.conversation_history = []
    
    def run(self, task):
        """Execute agent loop"""
        
        print(f"🎯 Task: {task}\n")
        
        self.conversation_history = [
            {"role": "user", "content": task}
        ]
        
        for iteration in range(self.max_iterations):
            print(f"--- Iteration {iteration + 1} ---")
            
            # 1. THINK: Get agent's next action
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=self.conversation_history,
                tools=self.tools
            )
            
            response_message = response.choices[0].message
            
            # 2. CHECK: Is task complete?
            if not response_message.tool_calls:
                print(f"\n✅ Task Complete!")
                return response_message.content
            
            # 3. ACT: Execute tool calls
            self.conversation_history.append(response_message)
            
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"  🔧 Action: {function_name}({function_args})")
                
                # Execute function
                function_to_call = available_functions[function_name]
                
                if function_args:
                    result = function_to_call(**function_args)
                else:
                    result = function_to_call()
                
                print(f"  📊 Observation: {result}")
                
                # 4. OBSERVE: Add result to history
                self.conversation_history.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result) if isinstance(result, dict) else str(result)
                })
            
            print()
        
        return "⚠️ Max iterations reached. Task incomplete."

print("✅ SimpleAgent class defined")

In [ ]:
# Test the agent

agent = SimpleAgent(tools=agent_tools, max_iterations=5)

# Test with a multi-step task
result = agent.run("What's the refund policy and what's today's date?")

print("\n" + "="*60)
print(f"🤖 Final Answer: {result}")
print("="*60)

## Part 4: Memory Systems

Agents need memory to maintain context across interactions.

In [ ]:
class ConversationMemory:
    """Manages conversation history with sliding window"""
    
    def __init__(self, max_messages=20):
        self.messages = []
        self.max_messages = max_messages
    
    def add_message(self, role, content):
        """Add a message to history"""
        self.messages.append({
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        })
        
        # Keep only recent messages
        if len(self.messages) > self.max_messages:
            # Keep system message + recent messages
            system_msgs = [m for m in self.messages if m['role'] == 'system']
            recent = self.messages[-(self.max_messages - len(system_msgs)):]
            self.messages = system_msgs + recent
    
    def get_messages(self):
        """Get messages for API (without timestamps)"""
        return [{"role": m['role'], "content": m['content']} 
                for m in self.messages]
    
    def get_conversation_summary(self):
        """Get summary of conversation"""
        return {
            "total_messages": len(self.messages),
            "user_messages": len([m for m in self.messages if m['role'] == 'user']),
            "assistant_messages": len([m for m in self.messages if m['role'] == 'assistant'])
        }

# Test conversation memory
memory = ConversationMemory(max_messages=5)

memory.add_message("user", "Hello")
memory.add_message("assistant", "Hi! How can I help?")
memory.add_message("user", "What's the weather?")
memory.add_message("assistant", "It's sunny!")

print("Conversation Summary:")
print(json.dumps(memory.get_conversation_summary(), indent=2))

print("\nMessages:")
print(json.dumps(memory.get_messages(), indent=2))

In [ ]:
class CustomerMemory:
    """Manages customer profile and interaction history"""
    
    def __init__(self):
        self.profiles = {}  # In production: use database
    
    def get_profile(self, customer_id):
        """Get customer profile"""
        if customer_id not in self.profiles:
            # Load from mock database
            self.profiles[customer_id] = self._load_from_db(customer_id)
        
        return self.profiles[customer_id]
    
    def update_profile(self, customer_id, updates):
        """Update customer preferences"""
        profile = self.get_profile(customer_id)
        profile.update(updates)
        # In production: save to database
    
    def _load_from_db(self, customer_id):
        """Mock database load"""
        # Mock data
        mock_db = {
            "CUST-001": {
                "customer_id": "CUST-001",
                "name": "John Smith",
                "email": "john@email.com",
                "tier": "Gold",
                "past_orders": ["ORD-123", "ORD-456"],
                "preferences": {
                    "communication": "email",
                    "language": "en"
                },
                "past_issues": [
                    {
                        "date": "2024-11-15",
                        "issue": "Late delivery",
                        "resolved": True
                    }
                ]
            },
            "CUST-002": {
                "customer_id": "CUST-002",
                "name": "Jane Doe",
                "email": "jane@email.com",
                "tier": "Silver",
                "past_orders": ["ORD-789"],
                "preferences": {
                    "communication": "sms",
                    "language": "en"
                },
                "past_issues": []
            }
        }
        
        return mock_db.get(customer_id, {
            "customer_id": customer_id,
            "name": "Unknown",
            "tier": "Bronze"
        })

# Test customer memory
customer_memory = CustomerMemory()

profile = customer_memory.get_profile("CUST-001")
print("Customer Profile:")
print(json.dumps(profile, indent=2))

## Part 5: SupportGenie v0.5 - Autonomous Agent

Combining everything: agent loop + memory + tools

In [ ]:
class SupportGenieV5:
    """
    SupportGenie Version 0.5
    Autonomous agent with memory and multi-step reasoning
    """
    
    SYSTEM_MESSAGE = """You are SupportGenie, an autonomous customer support agent.
    
    You can:
    - Look up order status
    - Create support tickets
    - Check customer accounts
    - Search the knowledge base
    
    When given a task:
    1. Break it down into steps
    2. Use tools as needed
    3. Remember context from conversation
    4. Adapt based on results
    5. Confirm completion with user
    
    Be proactive and helpful. If you need more information, ask clearly."""
    
    def __init__(self, api_key):
        self.client = OpenAI(api_key=api_key)
        self.conversation_memory = ConversationMemory()
        self.customer_memory = CustomerMemory()
        self.setup_tools()
    
    def setup_tools(self):
        """Define available tools"""
        self.tools = [
            {
                "type": "function",
                "function": {
                    "name": "get_order_status",
                    "description": "Get order status and tracking",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "order_id": {"type": "string"}
                        },
                        "required": ["order_id"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "create_support_ticket",
                    "description": "Create a support ticket",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "subject": {"type": "string"},
                            "description": {"type": "string"},
                            "priority": {
                                "type": "string",
                                "enum": ["low", "medium", "high", "urgent"]
                            }
                        },
                        "required": ["subject", "description"]
                    }
                }
            },
            {
                "type": "function",
                "function": {
                    "name": "search_knowledge_base",
                    "description": "Search knowledge base for information",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {"type": "string"}
                        },
                        "required": ["query"]
                    }
                }
            }
        ]
        
        self.available_functions = {
            "get_order_status": self.get_order_status,
            "create_support_ticket": self.create_support_ticket,
            "search_knowledge_base": search_knowledge_base
        }
    
    def get_order_status(self, order_id: str) -> dict:
        """Mock order lookup"""
        orders = {
            "ORD-123": {"status": "delivered", "date": "2024-12-20"},
            "ORD-456": {"status": "shipped", "tracking": "1Z999AA10203"},
            "ORD-789": {"status": "processing"}
        }
        return orders.get(order_id, {"error": "Order not found"})
    
    def create_support_ticket(self, subject: str, description: str, priority: str = "medium") -> dict:
        """Mock ticket creation"""
        ticket_id = f"TKT-{random.randint(10000, 99999)}"
        return {
            "ticket_id": ticket_id,
            "subject": subject,
            "priority": priority,
            "status": "open"
        }
    
    def chat(self, user_message: str, customer_id: str = None) -> str:
        """Chat with autonomous agent"""
        
        # Load customer context if available
        context = ""
        if customer_id:
            profile = self.customer_memory.get_profile(customer_id)
            context = f"\n\nCustomer Profile:\n{json.dumps(profile, indent=2)}"
        
        # Add to conversation memory
        full_message = user_message + context
        self.conversation_memory.add_message("user", full_message)
        
        # Run agent loop (max 10 iterations)
        for iteration in range(10):
            messages = [
                {"role": "system", "content": self.SYSTEM_MESSAGE}
            ] + self.conversation_memory.get_messages()
            
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=messages,
                tools=self.tools
            )
            
            response_message = response.choices[0].message
            
            # No more actions needed
            if not response_message.tool_calls:
                self.conversation_memory.add_message(
                    "assistant",
                    response_message.content
                )
                return response_message.content
            
            # Execute tool calls
            self.conversation_memory.messages.append(response_message)
            
            for tool_call in response_message.tool_calls:
                function_name = tool_call.function.name
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"  🔧 {function_name}({function_args})")
                
                # Execute
                result = self.available_functions[function_name](**function_args)
                
                # Add to memory
                self.conversation_memory.messages.append({
                    "role": "tool",
                    "tool_call_id": tool_call.id,
                    "content": json.dumps(result)
                })
        
        return "Task taking longer than expected. Would you like me to create a ticket for human review?"

print("✅ SupportGenie v0.5 class defined")

In [ ]:
# Test SupportGenie v0.5

agent = SupportGenieV5(api_key=os.environ.get('OPENAI_API_KEY'))

print("🤖 SupportGenie v0.5 - Autonomous Agent with Memory\n")
print("="*60)

# Test 1: Simple query
print("\n👤 User: What's your refund policy?")
response = agent.chat("What's your refund policy?")
print(f"🤖 Agent: {response}")

print("\n" + "="*60)

# Test 2: Multi-step with customer context
print("\n👤 User: Check my order ORD-123")
response = agent.chat("Check my order ORD-123", customer_id="CUST-001")
print(f"🤖 Agent: {response}")

print("\n" + "="*60)

# Test 3: Conversational memory (follow-up)
print("\n👤 User: And what about the warranty?")
response = agent.chat("And what about the warranty?")
print(f"🤖 Agent: {response}")

## Part 6: Advanced Agent Patterns

### Pattern 1: Plan-and-Execute

In [ ]:
def create_plan(task: str) -> list:
    """
    Create a plan for completing a task
    """
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[{
            "role": "user",
            "content": f"""Break down this task into 3-5 clear steps:
            
            Task: {task}
            
            Return as a numbered list."""
        }]
    )
    
    return response.choices[0].message.content

# Test planning
task = "Help a customer who received a damaged product and wants a refund"
plan = create_plan(task)

print("📋 Plan:")
print(plan)

## Exercises

### Exercise 1: Add Memory Persistence

Modify ConversationMemory to save/load from a JSON file.

In [ ]:
# Your code here
class PersistentMemory(ConversationMemory):
    def save_to_file(self, filename):
        """Save conversation to file"""
        # TODO: Implement file saving
        pass
    
    def load_from_file(self, filename):
        """Load conversation from file"""
        # TODO: Implement file loading
        pass

### Exercise 2: Implement Self-Critique

Add a method for the agent to review its own responses.

In [ ]:
# Your code here
def critique_response(query: str, response: str) -> dict:
    """
    Agent critiques its own response
    Returns: {"score": 1-10, "improvements": "..."}
    """
    # TODO: Implement self-critique
    pass

### Exercise 3: Build a Research Agent

Create an agent that can search multiple sources and synthesize information.

In [ ]:
# Your code here
class ResearchAgent:
    """
    Agent that researches a topic across multiple sources
    """
    # TODO: Implement research agent
    pass

## 🎉 Session 5 Complete!

### What You Learned:

✅ Agents autonomously complete multi-step tasks  
✅ ReAct pattern: Reason → Act → Observe → Repeat  
✅ Memory is critical for context  
✅ Always limit iterations  
✅ Handle errors gracefully  
✅ SupportGenie is now truly autonomous!

### Next Steps:

In **Session 6: Multi-Agent Orchestration**, you'll learn:
- Router agents for query classification
- Specialist agents for different domains
- Orchestration patterns (sequential, parallel, hierarchical)
- Building complex multi-agent systems

---

**Continue to**: [Session 6: Multi-Agent →](06_Multi_Agent.ipynb)